# Pandas & SQL : Practice

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlite3

### Example 1

Write a query that returns the user ID of all users that have created at least one ‘Refinance’ 
submission and at least one ‘InSchool’ submission."

In [3]:
data ={
    'user_id' : [1,2,2,4,2,6,7,8,4,4],
    'submisson' : ['yes','inschool', 'refinance','inschool','no','yes','no','refinance','no','inschool']
}

df = pd.DataFrame(data)
df

,user_id,submisson
0,1,yes
1,2,inschool
2,2,refinance
3,4,inschool
4,2,no
5,6,yes
6,7,no
7,8,refinance
8,4,no
9,4,inschool


In [4]:
conn = sqlite3.connect(":memory:")
df.to_sql('tab', conn)

10

In [5]:
myquery="""
    SELECT 
    user_id,
    COUNT(*) as row_count
    FROM tab
    WHERE submisson IN ('inschool','refinance')
    GROUP BY user_id
"""

pd.read_sql(myquery, conn)

,user_id,row_count
0,2,2
1,4,2
2,8,1


In [6]:
myquery="""
    SELECT 
    user_id,
    COUNT(*) as row_count
    FROM tab
    WHERE submisson IN ('inschool','refinance')
    GROUP BY user_id
    HAVING row_count=2
    """

pd.read_sql(myquery, conn)

,user_id,row_count
0,2,2
1,4,2


### Example 2

Find duplicate values in a SQL table

In [7]:
data = [
    [1,'John','asd@asd.com'],
    [2,'Sam','asd@asd.com'],
    [3,'Tom','asd@asd.com'],
    [4,'Bob','bob@asd.com'],
    [5,'Tom','asd@asd.com']
]
df =  pd.DataFrame(data, columns=['ID','NAME','EMAIL'])
df

,ID,NAME,EMAIL
0,1,John,asd@asd.com
1,2,Sam,asd@asd.com
2,3,Tom,asd@asd.com
3,4,Bob,bob@asd.com
4,5,Tom,asd@asd.com


In [8]:
conn = sqlite3.connect(":memory:")
df.to_sql('tab', conn)

5

In [9]:
myquery="""
    SELECT 
    name,
    email,
    COUNT(*) as row_count
    FROM tab
    GROUP BY name, email
    HAVING row_count>1
    """

pd.read_sql(myquery, conn)

,NAME,EMAIL,row_count
0,Tom,asd@asd.com,2


### Example 3

Given a Weather table, write a SQL query to find all dates' Ids with higher temperature compared to its previous (yesterday's) dates.

In [10]:
data = [
    [1,'2015-01-01', 10],
    [2,'2015-01-02', 25],
    [3,'2015-01-03', 20],
    [4,'2015-01-04', 30]]
df =  pd.DataFrame(data, columns=['Id','RecordDate','Temp'])
df

,Id,RecordDate,Temp
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [11]:
conn = sqlite3.connect(":memory:")
df.to_sql('tab', conn)

4

In [12]:
myquery="""
    WITH f1 AS
    (SELECT 
    id,
    RecordDate,
    CAST(Temp as INT) as Temp,
    LAG(CAST(Temp as INT),1) OVER () as Lag_Temp
    FROM tab)
    SELECT 
    *,
    (Temp-Lag_Temp) as Temp_Diff
    FROM f1
    """

pd.read_sql(myquery, conn)

,id,RecordDate,Temp,Lag_Temp,Temp_Diff
0,1,2015-01-01,10,NaN,NaN
1,2,2015-01-02,25,10.0,15.0
2,3,2015-01-03,20,25.0,-5.0
3,4,2015-01-04,30,20.0,10.0


In [13]:
myquery="""
    WITH f1 AS
    (SELECT 
    id,
    RecordDate,
    CAST(Temp as INT) as Temp,
    LAG(CAST(Temp as INT),1) OVER () as Lag_Temp
    FROM tab)
    SELECT 
    f1.id,
    (Temp-Lag_Temp) as Temp_Diff
    FROM f1
    WHERE Temp_Diff > 0
    """

pd.read_sql(myquery, conn)

,id,Temp_Diff
0,2,15
1,4,10


### Example 4

Select rows randomly from a  table  

In [14]:
myquery="""
    SELECT
    *
    FROM tab
    ORDER BY RANDOM()
    """

pd.read_sql(myquery, conn)

,index,Id,RecordDate,Temp
0,3,4,2015-01-04,30
1,1,2,2015-01-02,25
2,2,3,2015-01-03,20
3,0,1,2015-01-01,10


### Example 5

Exchange rows

In [15]:
data = [
    [1,'Abbot'],
    [2,'Doris'],
    [3,'Emerson'],
    [4,'Green'],
    [5,'Jeames']]
df =  pd.DataFrame(data, columns=['Id','Name'])
df

,Id,Name
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


In [16]:
conn = sqlite3.connect(":memory:")
df.to_sql('tab', conn)

5

In [17]:
myquery="""
    SELECT
    Id,
    Name,
    CASE 
    WHEN (SELECT Max(Id) FROM tab)%2=1 AND Id=(SELECT Max(Id) FROM tab) THEN Id
    WHEN Id%2=1 THEN Id+1
    ELSE Id-1
    END as Swapped_Id
    FROM tab
    ORDER BY Swapped_Id
    """

pd.read_sql(myquery, conn)

,Id,Name,Swapped_Id
0,2,Doris,1
1,1,Abbot,2
2,4,Green,3
3,3,Emerson,4
4,5,Jeames,5
